# Dropout Regularization in Deep Learning Models With Keras

Adaptado de https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

El Dropout es una técnica de regularización simple y poderosa que se aplica en Redes Neuronales y Deep Learning.

Los objetivos de este ejercicio son:

- Ver como funciona la técnica de dropout en Keras
- Descripción del modelo a usar
- Como usar dropout en la capa de entrada
- Como usar dropout en las capas hidden
- Notas para el uso de dropout


### Como utilizar Dropout en Keras

Ver el paper original de Srivastava y otros (2014): [A Simple Way to Prevent Neural Networks from Overfitting](http://jmlr.org/papers/v15/srivastava14a.html)

En Keras el dropout se implementa facilmente seleccionando los nodos que serán ignorados (con una probabilidad dada) en el ciclo de actualización de los pesos.

El Dropout se utiliza solamente en la fase de entrenamiento del modelo. Por lo tanto no se aplica cuando se evalúa la capacidad del modelo.


### Descripción del modelo a usar

Para el ejemplo usaremos el [dataset de Sonar](http://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/), que representa un problema de **clasificación binaria** donde hay que clasificar rocas e imitaciones de rocas según la señal devuelta por un sonar. Este es un buen dataset para NN ya que la mayoría de sus datos son numéricos y están a escala.

Se implementará un modelo basado en scikit-learn con una cross validation 10-fold, y se evaluaran los resultados de la red.

Por cada muestra de datos hay 60 features de entrada y 1 único valor de salida, y todos están normalizados antes de ser usados por la NN. Las funciones de activación en las capas hidden serán **relu** y la de salida será (por supuesto) una **sigmoide**.

El modelo de NN de referencia tiene dos capas hidden, la primera con 60 unidades y la segunda con 30. Se utilizará SGD para entrenar el modelo con un **learning-rate** y **momentum** relativamente bajos.


In [1]:
# Cargamos las librerias relevantes
import numpy
from pandas import read_csv

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Using Theano backend.


### Cargamos los datos

In [2]:
numpy.random.seed(7)

# Cargamos el dataset
dataframe = read_csv("data/sonar.csv", header=None)
dataset = dataframe.values

# separamos los predictores (X) de los targets (Y)
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# codificamos los valores categoricos como enteros
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [3]:
# Verificamos las dimensiones de los datos
print X.shape
print Y.shape

(208, 60)
(208,)


In [4]:
# Como se ven los datos de una muestra
X[1]

array([ 0.0453,  0.0523,  0.0843,  0.0689,  0.1183,  0.2583,  0.2156,
        0.3481,  0.3337,  0.2872,  0.4918,  0.6552,  0.6919,  0.7797,
        0.7464,  0.9444,  1.    ,  0.8874,  0.8024,  0.7818,  0.5212,
        0.4052,  0.3957,  0.3914,  0.325 ,  0.32  ,  0.3271,  0.2767,
        0.4423,  0.2028,  0.3788,  0.2947,  0.1984,  0.2341,  0.1306,
        0.4182,  0.3835,  0.1057,  0.184 ,  0.197 ,  0.1674,  0.0583,
        0.1401,  0.1628,  0.0621,  0.0203,  0.053 ,  0.0742,  0.0409,
        0.0061,  0.0125,  0.0084,  0.0089,  0.0048,  0.0094,  0.0191,
        0.014 ,  0.0049,  0.0052,  0.0044])

In [5]:
# Creacion de la red neuronal
def crea_NN():
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compila el modelo
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

### Entrenamiento del Modelo básico

In [7]:
seed = 7
numpy.random.seed(7)
estimators = []
estimators.append(('standardize', StandardScaler()))

estimators.append(('mlp', KerasClassifier(build_fn=crea_NN, 
                                          epochs=300, 
                                          batch_size=16, 
                                          verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named mkl


In [8]:
print("Accuracy de la clasificación (sin dropout): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy de la clasificación (sin dropout): 85.02% (6.04%)


### Como usar dropout en la capa de entrada

El Dropout puede aplicarse a las neuronas de la capa de entrada o capa visible.

En el ejemplo de abajo, agregamos una nueva capa de dropout entre la capa de entrada y la primera capa hidden.  

Se asigna un valor de 0.2 para el dropout, lo que significa que el 20% de los valores de entrada serán aleatoriamente excluídas en cada ciclo de actualización.

Adicionalmente, como se recomienda en al paper original, se impone una restricción sobre los pesos de cada capa oculta, asegurando que la norma máxima de los pesos no exceda un valor de 3. Esto se hace estableciendo el argumento *kernel_constraint* en la clase Dense al construir las capas.

El "learning-rate" ha sido aumentado en un orden de magnitud y el valor del "momentum" ha sido incrementado a 0.9. Estos cambios fueron recomendados en el paper original de dropout.

In [9]:
# Dropout en la capa de entrada with weight constraint
def crea_NN_dropout_1():
    # Creamos el modelo
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compilamos el modelo
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [10]:
seed=7

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=crea_NN_dropout_1,
                                          epochs=300, 
                                          batch_size=16,
                                          verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)


In [11]:
print("Accuracy de la clasificación (con dropout capa de entrada): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy de la clasificación (con dropout capa de entrada): 86.06% (4.52%)


###  Como usar dropout en las capas hidden

El Dropout puede ser aplicado a las neuronas de las capas hidden dentro de la red neuronal.

En el ejemplo de abajo, el dropout se aplica entre las 2 capas hidden y entre la última capa hidden y la capa de salida. De nuevo, se aplica un valor de dropout del 20%, así como una restricción de peso sobre dichas capas.


In [12]:
# Dropout en capas hidden con constricción de pesos
def crea_NN_dropout_hidden():
    # Creamos el modelo
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compilar modelo
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [13]:
seed=7

numpy.random.seed(seed)

estimators = []

estimators.append(('standardize', StandardScaler()))

estimators.append(('mlp', KerasClassifier(build_fn=crea_NN_dropout_hidden, 
                                          epochs=300, 
                                          batch_size=16, 
                                          verbose=0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

In [14]:
print("Accuracy de la clasificación (con dropout en capas hidden): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy de la clasificación (con dropout en capas hidden): 83.09% (5.96%)


Vemos que, para este problema y para esta configuración de NN, usando dropout en las capas hidden no mejora la performance. De hecho ha sido peor que en el caso sin dropout.

Es posible que el training requiera más epochs o que se necesite otro tuning en "learning rate".

### Notas para el uso de Dropout

El paper original de Dropout proporciona resultados experimentales en un conjunto de problemas típicos de machine learning. El paper proporciona un numero de heuristicas útiles a tener en cuenta cuando usamos dropout en la práctica.

- **Usar valores de dropout de 20%-50%** (20% es un buen comienzo). Una probabilidad demasiada baja tiene un efecto mínimo y un valor demasiado alto impedirá que la red aprenda adecuadamente.

- **Usar una red grande.** Se obtendrán mejores resultados con una red 'grande', dandole al modelo más opciones de aprender representaciones independientes.

- **Usar dropout en las capas de entrada y en las hidden.** Aplicar el dropout en todas las capas de la NN ha mostrado buenos resultados.

- **Usar valores grandes de "learning rate" con "decay" y también un valor alto de "momentum".** Incrementar el  learning rate por un a factor de 10 a 100 y usar valores de momentum de 0.9 o 0.99.

- **Limitar los pesos de la NN**. Un  learning rate alto puede generar pesos muy grandes en la red. Imponer un límite en los pesos (tal como la regularización max-norm con un tamaño de 4 o 5) ha mostrado una mejora en los resultados.